In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [5]:
import requests
import json

MODEL = 'llama3'  
OLLAMA_URL = 'http://localhost:11434/api/generate'

prompt = "What's the capital of France?"

# Construct request payload
data = {
    "model": MODEL,
    "prompt": prompt,
    "stream": False  # Set to True for streaming responses
}

response = requests.post(OLLAMA_URL, json=data)

if response.status_code == 200:
    output = response.json()
    print("Response:", output.get("response"))
else:
    print(f"Error: {response.status_code}")
    print(response.text)



Response: The capital of France is Paris!


In [10]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [7]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [11]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [12]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [14]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [15]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-R1-0528',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/ResembleAI/chatterbox',
 '/Qwen/Qwen3-Embedding-0.6B-GGUF',
 '/deepseek-ai/DeepSeek-R1-0528-Qwen3-8B',
 '/models',
 '/spaces/ResembleAI/Chatterbox',
 '/spaces/enzostvs/deepsite',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/alexnasa/Chain-of-Zoom',
 '/spaces/NihalGazi/Text-To-Speech-Unlimited',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/yandex/yambda',
 '/datasets/open-thoughts/OpenThoughts3-1.2M',
 '/datasets/open-r1/Mixture-of-Thoughts',
 '/datasets/Hcompany/WebClick',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Wri

In [16]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [18]:
import requests
import json
from bs4 import BeautifulSoup
import urllib.request

MODEL = 'llama3'  # Your ollama model
OLLAMA_URL = 'http://localhost:11434/api/generate'

def get_links_user_prompt(website_content):
    return f"Extract all hyperlinks from the following HTML content and return them as a JSON list:\n\n{website_content}"

def get_links(url):
    # Get raw HTML
    response = urllib.request.urlopen(url)
    html = response.read().decode('utf-8')

    # Construct Ollama request
    data = {
        "model": MODEL,
        "prompt": get_links_user_prompt(html),
        "stream": False
    }

    # Call Ollama
    res = requests.post(OLLAMA_URL, json=data)
    if res.status_code == 200:
        output = res.json()
        try:
            return json.loads(output['response'])  # assuming the model returns JSON string
        except json.JSONDecodeError:
            print("Model output was not valid JSON:", output['response'])
            return {"links": []}
    else:
        print("Ollama error:", res.text)
        return {"links": []}


In [19]:
print(get_all_details("https://huggingface.co"))


Model output was not valid JSON: Here are the hyperlinks extracted from the HTML content in JSON format:

```
[
  {
    "href": "/models",
    "text": "Models"
  },
  {
    "href": "/datasets",
    "text": "Datasets"
  },
  {
    "href": "/spaces",
    "text": "Spaces"
  },
  {
    "href": "/changelog",
    "text": "Changelog"
  },
  {
    "href": "https://endpoints.huggingface.co",
    "text": "Inference Endpoints"
  },
  {
    "href": "/chat",
    "text": "HuggingChat"
  },
  {
    "href": "/huggingface",
    "text": "About"
  },
  {
    "href": "/brand",
    "text": "Brand assets"
  },
  {
    "href": "/terms-of-service",
    "text": "Terms of service"
  },
  {
    "href": "/privacy",
    "text": "Privacy"
  },
  {
    "href": "https://apply.workable.com/huggingface/",
    "text": "Jobs"
  },
  {
    "href": "mailto:press@huggingface.co",
    "text": "Press"
  },
  {
    "href": "/learn",
    "text": "Learn"
  },
  {
    "href": "/docs",
    "text": "Documentation"
  },
  {
    "hre

In [20]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [21]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))